In [1]:
%load_ext autoreload
%autoreload 2 

import os
import numpy as np
import pickle

from tqdm import tqdm

import torch
import torch.utils.data
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

from collections import OrderedDict

import nltk
from nltk.tokenize import word_tokenize

from embeddings import load_embeddings, load_vocab
from load_conll import load_conll03
from loader import prepare_sentence, tag_mapping, cap_feature, CoNLLDataset, pad_list
from model import Tagger, cuda
from torch_utils import prepare_sequence, prepare_sequence_float, tensor
from utils import sent2seq, sent2chars, word_index, char_index, add_unknown_last, zero_digits
from eval import eval, micro_precision_recall_f1_accuracy, eval_metrics, eval_metrics_crf, save_plot

Using TensorFlow backend.


In [2]:
#Parameters
torch.manual_seed(0)

parameters = OrderedDict()

parameters["lr"] = 0.1
parameters["optimizer"] = "SGD"
parameters["hidden_size"] = 200
parameters["pre_emb"] = "glove"

parameters["w_embed_size"] = 300
# parameters["dim_cap"] = 10

parameters["batch_size"] = 20
parameters["load_embeds"] = True
parameters["dropout"] = 0.5
parameters["gradient_clipping"] = 0
parameters["crf"] = True

epochs = 1000
zero_digit = True

assert parameters["pre_emb"] in ["glove", "google"]
assert not parameters["pre_emb"] == "google" or parameters["w_embed_size"] == 300

param_str = "-".join(["%s:%s" % (str(k), str(v)) for (k,v) in parameters.items()]).lower()
print(param_str)

lr:0.1-optimizer:sgd-hidden_size:200-pre_emb:glove-w_embed_size:300-batch_size:20-load_embeds:true-dropout:0.5-gradient_clipping:0-crf:true


# 1. Data Preprocessing

### Load pretrained embeddings

In [3]:
if parameters["pre_emb"] == "glove":
    embeddings_path = "word_embeddings/glove.6B/glove.6B.%sd_w2vformat.txt" % parameters["w_embed_size"]
    binary = False
else:
    embeddings_path = "word_embeddings/google/GoogleNews-vectors-negative300.bin"
    binary = True
    
if parameters["load_embeds"]:
    loaded_embeddings, (w2idx, idx2w) = load_embeddings(embeddings_path, binary=binary)
else:
    parameters["freeze"] = 0

Loading from saved embeddings
Loading vocab


### Load CoNLL

In [4]:
# CoNLL03
sents_train_03, pos_train_03, chunk_train_03, ner_train_03 = load_conll03(["cleaned_eng.train"])
sents_dev_03, pos_dev_03, chunk_train_03, ner_dev_03 = load_conll03(["cleaned_eng.testa"])
sents_test_03, pos_test_03, chunk_train_03, ner_test_03 = load_conll03(["cleaned_eng.testb"])

print("Train %s, Dev %s, Test %s" % (len(sents_train_03), len(sents_dev_03), len(sents_test_03)))

Loaded CoNLL03 in 1.8718538284301758 seconds
Loaded CoNLL03 in 0.43241214752197266 seconds
Loaded CoNLL03 in 0.4268953800201416 seconds
Train 14041, Dev 3250, Test 3453


In [5]:
if zero_digit:
    sents_train_03 = [[zero_digits(w) for w in s] for s in sents_train_03]
    sents_test_03 = [[zero_digits(w) for w in s] for s in sents_test_03]    
    sents_dev_03 = [[zero_digits(w) for w in s] for s in sents_dev_03]    

In [6]:
sents_train = np.concatenate([sents_train_03, sents_dev_03, sents_test_03])

In [7]:
w2idx_train, idx2w_train = word_index(sents_train)
w2idx_train, idx2w_train = add_unknown_last(w2idx_train, idx2w_train)

X_train_03 = sent2seq(sents_train_03, w2idx_train)
X_dev_03 = sent2seq(sents_dev_03, w2idx_train)
X_test_03 = sent2seq(sents_test_03, w2idx_train)

idner_train, ner2idx, idx2ner = tag_mapping(ner_train_03)
idner_dev = tag_mapping(ner_dev_03, ner2idx)
idner_test = tag_mapping(ner_test_03, ner2idx)

num_ner_classes = len(ner2idx)

Found 9 unique named entity tags


### Data Pipeline

In [8]:
class CoNLLDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, lens):
        self.data = X
        self.labels = y
        self.lens = lens
        
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx], self.lens[idx]

    def __len__(self):
        return len(self.data)

In [9]:
def pad_list(x, padding_idx=0):
    lens = [len(s) for s in x]
    maxlen = max(lens)    
    sorted_indices = sorted(range(len(lens)), key=lambda k: lens[k], reverse=True)
    
    batch = padding_idx * torch.ones(len(x), maxlen).long()
    
    for i, idx in enumerate(sorted_indices):
        batch[i, :lens[idx]] = torch.LongTensor(x[idx])
    
    return batch, sorted(lens, reverse=True), sorted_indices

In [10]:
data_train, lens_train, sorted_train = pad_list(X_train_03)
labels_train, _, _ = pad_list(idner_train)

data_dev, lens_dev, sorted_dev = pad_list(X_dev_03)
labels_dev, _, _ = pad_list(idner_dev)

data_test, lens_test, sorted_test = pad_list(X_test_03)
labels_test, _, _ = pad_list(idner_test)

In [11]:
dataset_train = CoNLLDataset(data_train, labels_train, lens_train)
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=parameters["batch_size"], num_workers=4)

dataset_dev = CoNLLDataset(data_dev, labels_dev, lens_dev)
loader_dev = torch.utils.data.DataLoader(dataset_dev, batch_size=parameters["batch_size"], num_workers=4)

dataset_test = CoNLLDataset(data_test, labels_test, lens_test)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=parameters["batch_size"], num_workers=4)

In [12]:
idner_dev = [idner_dev[i] for i in sorted_dev]
idner_test = [idner_test[i] for i in sorted_test]

sents_dev_03 = [sents_dev_03[i] for i in sorted_dev]
sents_test_03 = [sents_test_03[i] for i in sorted_test]

In [13]:
# for x, y, l in loader:
#     packed = pack_padded_sequence(autograd.Variable(x), l.numpy(), batch_first=True)
#     batch, lens = pad_packed_sequence(packed, batch_first=True)

### Fit word embeddings to vocabulary

In [14]:
embeddings = np.random.normal(scale=0.001, size=(len(w2idx_train), parameters["w_embed_size"]))

if parameters["load_embeds"]:
    for w, i in w2idx_train.items():
        idx = w2idx.get(w)
        if idx is not None:
            embeddings[i] = loaded_embeddings[idx][:parameters["w_embed_size"]]

### Capitalization features

In [15]:
# cap_train_03 = [[cap_feature(w) for w in s] for s in sents_train_03]
# cap_test_03 = [[cap_feature(w) for w in s] for s in sents_test_03]
# cap_dev_03 = [[cap_feature(w) for w in s] for s in sents_dev_03]    

In [16]:
# if parameters["dim_cap"]:
#     n_cap = 4
#     cap_embeddings = np.random.normal(scale=0.001, size=(n_cap, parameters["dim_cap"]))

# 2. Training 

### Defining model

In [17]:
if not "<START>" in idx2ner.values():
    idx2ner[len(idx2ner)] = "<START>"
    idx2ner[len(idx2ner)] = "<STOP>"
    ner2idx = {v:k for (k,v) in idx2ner.items()}

idx2ner

{0: 'O',
 1: 'B-LOC',
 2: 'B-PER',
 3: 'B-ORG',
 4: 'I-PER',
 5: 'I-ORG',
 6: 'B-MISC',
 7: 'I-LOC',
 8: 'I-MISC',
 9: '<START>',
 10: '<STOP>'}

In [18]:
model = Tagger(tensor(embeddings),  parameters["hidden_size"], idx2ner, dropout=parameters["dropout"],
               crf=parameters["crf"])

trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())

if parameters["optimizer"].lower() == "adam":
    optimizer = optim.Adam(trainable_parameters, lr= parameters["lr"])
elif parameters["optimizer"].lower() == "sgd":
    optimizer = optim.SGD(trainable_parameters, lr= parameters["lr"])

if torch.cuda.is_available():
    model.cuda()

In [20]:
reload = 1
model_path = "models/ner_crf_batch/%s/" % param_str

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if reload and os.path.exists(model_path + "last_state_dict"):
    model.load_state_dict(torch.load(model_path + "last_state_dict"))
    model = model.cuda()
    with open(model_path + "metrics.p", "rb") as file:
        metrics = pickle.load(file)
    best_ner = np.max(metrics["ner"]["val_loss_dev"])
    
else:
    metrics = {"ner":{"precision":[], "recall":[], "f1":[], "accuracy":[], "ent_f1":[], "loss": [], "val_loss_dev": [],
                      "precision_test":[], "recall_test":[], "f1_test":[], "accuracy_test":[], "ent_f1_test":[], 
                      "val_loss_test": []}}
    best_ner = np.inf

### Training 

In [21]:
for epoch in range(epochs):
    print("Epoch %s/%s :" % (epoch+1, epochs))

    losses = []
    val_loss_epoch = []
    loss_epoch = []
    
    test_lim = 3500 
    for i, (sentences, tags, lens) in enumerate(tqdm(loader_train)):
#     test_lim = 10
#     for i in tqdm(range(10)):
                      
        sentences_in = autograd.Variable(cuda(sentences[:,:lens.numpy()[0]]))
        targets = autograd.Variable(cuda(tags[:,:lens.numpy()[0]]))
        
        packed_targets = pack_padded_sequence(targets, lens.numpy(), batch_first=True) 
        
        if parameters["crf"]:        
            loss = model.neg_log_likelihood(sentences_in, lens, targets,
                                            gradient_clipping=parameters["gradient_clipping"])
        else:
            scores = model(sentences_in, lens)
            padded_scores = pad_packed_sequence(scores, batch_first=True) 
            
#             print(padded_scores[0].size())
#             print(targets.size())
            
            loss = nn.CrossEntropyLoss()(padded_scores[0].contiguous().view(-1,len(ner2idx)),
                                                          targets.contiguous().view(-1))

        loss.backward()        
        optimizer.step()
        
        losses.append(loss.cpu().data.numpy())
    
    preds_dev, ner_loss_dev = model.test(loader_dev)  
    
    eval_metrics_crf(preds_dev, metrics, idner_dev[:test_lim], sents_dev_03[:test_lim], 
                     ner2idx, idx2ner, model_path, dev=True)
    
    val_loss_epoch = ner_loss_dev
    
    preds_test, ner_loss_test = model.test(loader_test)     
    
    eval_metrics_crf(preds_test, metrics, idner_test[:test_lim], sents_test_03[:test_lim], 
                         ner2idx, idx2ner, model_path)
    
    loss_epoch = np.mean(losses)
    
    print("Loss :  NER %s" % (loss_epoch))
    print("Dev loss : NER %s" % (val_loss_epoch))
    print("Test loss : NER %s" % (ner_loss_test))
    
    torch.save(model.state_dict(), model_path + "last_state_dict")
    if ner_loss_dev < best_ner: 
        print("New best score on dev.")
        print("Saving model...")
        torch.save(model.state_dict(), model_path + "best_state_dict")
        
        best_ner = ner_loss_dev
    
    metrics["ner"]["val_loss_dev"].append(val_loss_epoch)
    metrics["ner"]["val_loss_test"].append(ner_loss_test)    
    metrics["ner"]["loss"].append(loss_epoch)    

    # Save learning curve
    save_plot(metrics, model_path)
    with open(model_path + "metrics.p", "wb") as file:
        pickle.dump(metrics, file)

print("Done")

  0%|          | 0/703 [00:00<?, ?it/s]

Epoch 1/1000 :


100%|██████████| 703/703 [02:57<00:00,  3.96it/s]  
163it [00:18, 38.06it/s]


ner : p 0.976183486957, r 0.977771679473, f 0.976976937766, acc 0.9711849227055022
ID     NE  Total      O  B-LOC  B-PER  B-ORG  I-PER  I-ORG B-MISC  I-LOC I-MISC<START> <STOP>  Percent
 0      O  42759  42609     30     41     26      5     22     12      4     10      0      0   99.649
 1  B-LOC   1837     73   1692     27     30      2      4      6      3      0      0      0   92.107
 2  B-PER   1842    103     13   1666     43     13      3      1      0      0      0      0   90.445
 3  B-ORG   1341    105     40     64   1093      4     19     15      1      0      0      0   81.506
 4  I-PER   1307     50      0      7      0   1221     28      0      0      1      0      0   93.420
 5  I-ORG    751     59      5      3     21     16    622      3     12     10      0      0   82.823
 6 B-MISC    922     91     13     26     27      1      4    758      0      2      0      0   82.213
 7  I-LOC    257     10      2      0      0      6     17      0    221      1      0      0

173it [00:17,  9.87it/s]


...

ner : p 0.957436684906, r 0.957809558839, f 0.957623085576, acc 0.9533541509637127
ID     NE  Total      O  B-LOC  B-PER  B-ORG  I-PER  I-ORG B-MISC  I-LOC I-MISC<START> <STOP>  Percent
 0      O  38323  37993     37     56     64     22     54     43     17     37      0      0   99.139
 1  B-LOC   1668     90   1469     17     52      4      7     25      1      3      0      0   88.070
 2  B-PER   1617    242     24   1198    112     31      1      4      2      3      0      0   74.088
 3  B-ORG   1661    231     88     66   1212      2     28     34      0      0      0      0   72.968
 4  I-PER   1156     82      0      3      2   1020     42      4      2      1      0      0   88.235
 5  I-ORG    835     74      5      0     22     28    659      5     27     15      0      0   78.922
 6 B-MISC    702    131     19      7     18      0      2    519      0      6      0      0   73.932
 7  I-LOC    257     21      3      1      0      6     26      0    199      1      0      0

  0%|          | 0/703 [00:00<?, ?it/s]

Epoch 36/1000 :


 26%|██▌       | 183/703 [00:31<00:57,  9.02it/s]Process Process-424:
Process Process-423:
Process Process-421:
Process Process-422:
  File "/home/bruno/Apps/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 35, in _worker_loop
    r = index_queue.get()
  File "/home/bruno/Apps/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 35, in _worker_loop
    r = index_queue.get()
  File "/home/bruno/Apps/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 35, in _worker_loop
    r = index_queue.get()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/bruno/Apps/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/bruno/Apps/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/bruno/Apps/anaconda3/lib/python3.6/mul

KeyboardInterrupt: 

# Debug Tests

In [ ]:
for x,y,l in loader_train:
    X = autograd.Variable(x)
    Y = autograd.Variable(y)
    L = l
    break

In [ ]:
word_embeddings = nn.Embedding(len(w2idx_train), 300)

In [ ]:
packed = pack_padded_sequence(X, l.numpy(), batch_first=True)

In [ ]:
w = word_embeddings(X)

In [ ]:
w.size()

In [ ]:
p = pack_padded_sequence(w, l.numpy(), batch_first=True)

In [ ]:
p

In [ ]:
np.sum(p.batch_sizes)

In [ ]:
np.sum(l.numpy())

In [ ]:
print(l.numpy())
print(p.batch_sizes)